# Compute Results of Analysis

Combine SIFT, resnet, swin, vgg and vit features with basic distance analysis, query expansion and diffusion.

____
## Imports and Constants

In [36]:
import numpy as np
import pandas as pd
import os
import sklearn.metrics.pairwise
import my_eval
import query

###########################################

NOTEBOOK_DIR = "/home/sean/Code/Pawsey/4. Clean"

_____
## Load Data

In [4]:
## Load features

data = {}

for descriptor in os.listdir("./data"):

    if descriptor == "names":
        data[descriptor] = {"ox" : {}, "par" : {}}
        for fname in os.listdir("./data/" + descriptor):
            split_name = fname[:-4].split("-")
            dataset = split_name[0]
            if fname.endswith("y-names.npy"):
                data[descriptor][dataset]["y"] = np.load("./data/{}/{}".format(descriptor, fname))
            else:
                difficulty = split_name[2]
                data[descriptor][dataset][difficulty] = np.load("./data/{}/{}".format(descriptor, fname))

    else:
        data[descriptor] = {"ox" : {"E" : {}, "M" : {}, "H" :{}},
                            "par" : {"E" : {}, "M" : {}, "H" :{}}}
        for fname in os.listdir("./data/" + descriptor):
            split_name = fname[:-4].split("-")
            if len(split_name) == 3:
                pass
            else:
                (_, xy, dataset, difficulty) = split_name
                data[descriptor][dataset][difficulty][xy] = np.load("./data/{}/{}".format(descriptor, fname))
    
    print("Loaded " + descriptor)

data["swin"]["ox"]["E"]["x"]

Loaded swin
Loaded vgg
Loaded resnet
Loaded names
Loaded vit


array([[ 0.4381919 , -1.1369115 , -0.49100572, ..., -0.27456677,
         0.38102797, -0.30554023],
       [-0.19804995,  0.02098738,  0.52111053, ...,  0.44540596,
         0.8620084 ,  0.18907186],
       [ 1.0216093 , -0.06300209, -0.06569103, ...,  0.02202551,
        -0.32440802,  0.3858102 ],
       ...,
       [ 0.74518114, -0.9655011 , -0.55623275, ..., -0.39560622,
         0.3983633 , -0.4672271 ],
       [ 0.4493655 , -0.97439206, -0.61376625, ..., -0.19914342,
         0.27447924, -0.3482531 ],
       [ 0.04259995,  0.09633142,  0.65417933, ...,  0.5438953 ,
         0.53027916,  0.03832415]], dtype=float32)

In [27]:
q = data["swin"]["ox"]["E"]["y"]
t = data["swin"]["ox"]["E"]["x"]
q_n = data["names"]["ox"]["y"]
t_n = data["names"]["ox"]["E"]

_____
## Evaluate Methods

In [37]:
feature_type, dataset, difficulty, query_type, m_AP = [], [], [], [], []

for feature_type in ["swin", "vgg", "resnet", "vit"]:
    for dataset in ["ox", "par"]:
        for difficulty in ["E", "M", "H"]:
            queries = data[feature_type][dataset][difficulty]["x"]
            gallery = data[feature_type][dataset][difficulty]["y"]
            query_names = data["names"][dataset]["y"]
            gallery_names = data["names"][dataset][difficulty]

            ranks = query.return_ranks('basic', queries, gallery)
            mAP, precisions = my_eval.evaluate(ranks, query_names, gallery_names)
            print(mAP)

AttributeError: module 'query' has no attribute 'return_ranks'